In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import math
import statistics
import argparse
from scipy import signal
from scipy.signal import argrelextrema
from scipy.signal import savgol_filter
from scipy.fftpack import fft,ifft
from scipy.signal import butter, lfilter
from scipy.signal import freqs

In [ ]:
def do_mosaic(frame, x, y, w, h, neighbor=9):
  """
  马赛克的实现原理是把图像上某个像素点一定范围邻域内的所有点用邻域内左上像素点的颜色代替，这样可以模糊细节，但是可以保留大体的轮廓。
  :param frame: opencv frame
  :param int x : 马赛克左顶点
  :param int y: 马赛克右顶点
  :param int w: 马赛克宽
  :param int h: 马赛克高
  :param int neighbor: 马赛克每一块的宽
  """
  fh, fw = frame.shape[0], frame.shape[1]
  if (y + h > fh) or (x + w > fw):
    return
  for i in range(0, h - neighbor, neighbor): # 关键点0 减去neightbour 防止溢出
    for j in range(0, w - neighbor, neighbor):
      rect = [j + x, i + y, neighbor, neighbor]
      color = frame[i + y][j + x].tolist() # 关键点1 tolist
      left_up = (rect[0], rect[1])
      right_down = (rect[0] + neighbor - 1, rect[1] + neighbor - 1) # 关键点2 减去一个像素
      cv2.rectangle(frame, left_up, right_down, color, -1)

In [ ]:
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# Read video with OpenCV.
cap=cv2.VideoCapture('/home/johnson/Desktop/UPDRS/UPDRS_video/Tim_Tremor/t008_crop_256.mp4')

## Get video info
RES=(round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
fps = round(cap.get(cv2.CAP_PROP_FPS))

#create video writer to write detected_video
#output_video = "../UPDRS_result/t014_tremor.mp4"
#fourcc = cv2.VideoWriter_fourcc(*'MP4V')
#out = cv2.VideoWriter(output_video, fourcc, 30, RES)

In [ ]:
#store each frame in video
frame_count = 0
img_list = []
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        frame_count+=1
        img_list.append(frame)

    else:
        break
cap.release()


In [ ]:
#edit video to remove extra hands
frame_count = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        if 30 <= frame_count <= 300:
            out.write(frame)
        frame_count+=1

    else:
        break
cap.release()
out.release()

In [ ]:
#single hand (grasp)
hands = mp_hands.Hands(static_image_mode=True,max_num_hands=2,min_detection_confidence=0.7)

record = []
frame_count = 0
img_list = []
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        frame = frame[:,:128]
        frame_count += 1

        # Convert the BGR image to RGB, flip the image around y-axis for correct 
        # handedness output and process it with MediaPipe Hands.
        results = hands.process(cv2.flip(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 1))
        annotated_image = cv2.flip(frame.copy(), 1)

        if results.multi_hand_landmarks != None:
            for hand_landmarks in results.multi_hand_landmarks:
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                middle_y = hand_landmarks.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x
                record.append(middle_y)

                #after processing the hand, flip back the image
                annotated_image = cv2.flip(annotated_image, 1)
                img_list.append(annotated_image)
                #out.write(annotated_image)

    else:
        break

cap.release()
#out.release()
plt.plot(record)

In [ ]:
#multiple hand (pronation)
hands = mp_hands.Hands(static_image_mode=True,max_num_hands=4,min_detection_confidence=0.5)

history = {} #record which frame has detected result, 0 for detect fail, 1 for detect one hand, 2 for detect two hands
thumb_dis = []
thumb_list = []
frame_count = 0
img_list = []
results = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:

        history[str(frame_count)] = 0
        # Convert the BGR image to RGB, flip the image around y-axis for correct 
        # handedness output and process it with MediaPipe Hands.
        results = hands.process(cv2.flip(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 1))

        annotated_image = frame.copy()

        if results.multi_hand_landmarks != None:
            history[str(frame_count)] = 1
            annotated_image = cv2.flip(frame.copy(), 1)

            #collect all thumb's location
            temp = []
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                temp.append(hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y)
            thumb_list.append(temp)

            if len(results.multi_hand_landmarks) == 2:
                history[str(frame_count)] = 2

            #after processing the hand, flip back the image  
            annotated_image = cv2.flip(annotated_image, 1)

            '''
            if len(results.multi_hand_landmarks) == 2:
                if(results.multi_handedness[0].classification[0].label == results.multi_handedness[1].classification[0].label):
                    print(frame_count,results.multi_handedness[0].classification[0].label)
                else:
                    dis = abs(right_thumb-left_thumb)
                    thumb_dis.append(dis)
                    img_list.append(annotated_image)
            '''

        #out.write(annotated_image)
        img_list.append(annotated_image)
        frame_count += 1

    else:
        break

cap.release()
#out.release()

for thumb in thumb_list:
    if(len(thumb)) == 2:
        dis = abs(thumb[0]-thumb[1])
        thumb_dis.append(dis)
plt.plot(thumb_dis)



In [ ]:
#multiple hand
hands = mp_hands.Hands(static_image_mode=True,max_num_hands=4,min_detection_confidence=0.5)

record_left = []
record_right =[]
frame_count = 0
img_list = []
results = 0
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        frame_count += 1

        # Convert the BGR image to RGB, flip the image around y-axis for correct 
        # handedness output and process it with MediaPipe Hands.
        results = hands.process(cv2.flip(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB), 1))

        annotated_image = frame.copy()

        if results.multi_hand_landmarks != None:
            annotated_image = cv2.flip(frame.copy(), 1)
            for idx, hand_landmarks in enumerate(results.multi_hand_landmarks):
                mp_drawing.draw_landmarks(annotated_image, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                if results.multi_handedness[idx].classification[0].label == "Left":
                    thumb_y = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
                    record_left.append(thumb_y)
                elif results.multi_handedness[idx].classification[0].label == "Right":
                    thumb_y = hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y
                    record_right.append(thumb_y)

            #after processing the hand, flip back the image  
            annotated_image = cv2.flip(annotated_image, 1)
            img_list.append(annotated_image)

        #out.write(annotated_image)

    else:
        break

cap.release()
#out.release()



In [ ]:
fig, axs = plt.subplots(2)
axs[0].plot(record_right); axs[0].title.set_text("right hand")
axs[1].plot(record_left) ; axs[1].title.set_text("left hand")
fig.tight_layout()
#fig.savefig("UPDRS_result/pronation/patient_wave.png")

In [ ]:
copy = img_list[0][:,:128].copy()
#do_mosaic(copy, 300, 100, 350, 650,neighbor=30)
plt.imshow(cv2.cvtColor(copy, cv2.COLOR_BGR2RGB))
#plt.savefig("hand.png")

In [ ]:
copy = img_list[0].copy()
cv2.putText(copy,"action_count(right):",(100, 70), cv2.FONT_HERSHEY_SIMPLEX,
  1.5, (0, 0, 0), 3, cv2.LINE_AA)
cv2.putText(copy,"action_time(right):",(100, 120), cv2.FONT_HERSHEY_SIMPLEX,
  1.5, (0, 0, 0), 3, cv2.LINE_AA)
cv2.putText(copy,"action_count(left):",(1250, 70), cv2.FONT_HERSHEY_SIMPLEX,
  1.5, (0, 0, 0), 3, cv2.LINE_AA)
cv2.putText(copy,"action_time(left):",(1250, 120), cv2.FONT_HERSHEY_SIMPLEX,
  1.5, (0, 0, 0), 3, cv2.LINE_AA)
plt.imshow(cv2.cvtColor(copy, cv2.COLOR_BGR2RGB))

In [ ]:
smooth = record.copy()
for i in range(1,len(smooth)):
    smooth[i] = smooth[i-1]*0.5 + smooth[i]*0.5

time = np.linspace(0,frame_count/fps,frame_count)
plt.plot(time,smooth)


In [ ]:
def boxplot_outliers(data):
    # finding the 1st quartile
    q1 = np.quantile(data, 0.25)
    
    # finding the 3rd quartile
    q3 = np.quantile(data, 0.75)
    
    # finding the iqr region
    iqr = q3-q1
    
    # finding upper and lower whiskers
    upper_bound = q3+(1.5*iqr)
    lower_bound = q1-(1.5*iqr)
    print(lower_bound,upper_bound)

    outlier = [x for x in data if x > upper_bound]

    outlier_index = []
    for i in outlier:
        outlier_index.append(list(data).index(i))

    return outlier_index

In [ ]:
#find peaks of thumb distance(pronation)

smooth = thumb_dis.copy()
middle = (max(smooth)+min(smooth))/2

peaks, _ = signal.find_peaks(smooth,height=middle,prominence=0.1)


#calculcate fist closing action time
action_time_list = []
action_time_list.append(peaks[0])
for time in np.diff(peaks):
    action_time_list.append(time)

#show the peak wave
plt.plot(smooth)
plt.plot(peaks,np.array(smooth)[peaks],"x")
#plt.savefig('UPDRS_result/pronation/self_wave.png')

In [ ]:
#single hand
#smooth = record_right.copy()
#for i in range(1,len(smooth)):
#    smooth[i] = smooth[i-1]*0.5 + smooth[i]*0.5
after_filt = savgol_filter(record,31,5)

middle = (max(after_filt)+min(after_filt))/2
prominence = np.quantile(after_filt,0.75) - np.quantile(after_filt,0.25)

peaks, _ = signal.find_peaks(after_filt,height=middle,prominence=prominence)

fist_closing_frame = peaks

#calculcate fist closing action time
action_time_list = []
action_time_list.append(fist_closing_frame[0])
for time in np.diff(fist_closing_frame):
    action_time_list.append(time)

#show the peak wave
plt.plot(after_filt)
plt.plot(fist_closing_frame,np.array(after_filt)[fist_closing_frame],"x")
#plt.savefig('wave_normal.png')

In [ ]:
#multiple hand, 0 for right hand and 1 for left hand
fist_closing_frame = []
action_time_list = []
fig, axs = plt.subplots(2)

for hand in range(2):
    smooth = []
    label = ""
    if hand == 0:
        smooth = record_right.copy()
        label = "right hand"
    else:
        smooth = record_left.copy()
        label = "left hand"

    for i in range(1,len(smooth)):
        smooth[i] = smooth[i-1]*0.5 + smooth[i]*0.5
    middle = (max(smooth)+min(smooth))/2

    peaks, _ = signal.find_peaks(smooth,height=middle,prominence=0.05)

    fist_closing_frame.append(peaks)

    #calculcate fist closing action time
    temp = []
    temp.append(peaks[0])
    for time in np.diff(peaks):
        temp.append(time)
    action_time_list.append(temp)

    #show the peak wave
    #plt.plot(smooth)
    #plt.plot(peaks,np.array(smooth)[peaks],"x")
    #plt.savefig('wave_normal.png')

    axs[hand].plot(smooth) 
    axs[hand].plot(peaks,np.array(smooth)[peaks],"x")
    axs[hand].title.set_text(label)

fig.tight_layout()
fig.savefig('two-hands_wave.png')

In [ ]:
#write result to video (with image loss)(pronation)
action_count = 0
action_time = 0
two_hands_frame = 0
for i in range(len(img_list)):
    if history[str(i)] == 2:
        if len(peaks) > action_count and two_hands_frame == peaks[action_count]:
            action_time = action_time_list[action_count]
            action_count += 1
        two_hands_frame += 1

    cv2.putText(img_list[i],f'action_count:{action_count}',(10, 70), cv2.FONT_HERSHEY_SIMPLEX,1.5, (0, 0, 0), 3, cv2.LINE_AA)
    cv2.putText(img_list[i],f'action_time:{action_time/fps:.2f}s',(10, 120), cv2.FONT_HERSHEY_SIMPLEX,1.5, (0, 0, 0), 3, cv2.LINE_AA)
    out.write(img_list[i])

out.release()

In [ ]:
#write result to video (single hand)
action_count = 0
action_time = 0
for i in range(len(img_list)):
    if( len(fist_closing_frame) > action_count and i == fist_closing_frame[action_count]):
        action_time = action_time_list[action_count]
        action_count += 1
    cv2.putText(img_list[i],f'action_count:{action_count}',(10, 70), cv2.FONT_HERSHEY_SIMPLEX,1.5, (0, 0, 0), 3, cv2.LINE_AA)
    cv2.putText(img_list[i],f'action_time:{action_time/fps:.2f}s',(10, 120), cv2.FONT_HERSHEY_SIMPLEX,1.5, (0, 0, 0), 3, cv2.LINE_AA)
    out.write(img_list[i])

out.release()

In [ ]:
#write result to video (multiple hand)
action_count = [0,0]
action_time = [0,0]
for i in range(len(img_list)):
    label = ""
    text_loc_x = 0
    for hand in range(2):
        if hand == 0:
            label = "right"
            text_loc_x = 100
        else:
            label = "left"
            text_loc_x = 1250

        if( (len(fist_closing_frame[hand]) > action_count[hand]) and i == fist_closing_frame[hand][action_count[hand]]):
            action_time[hand] = action_time_list[hand][action_count[hand]]
            action_count[hand] += 1
        cv2.putText(img_list[i],f'action_count({label}):{action_count[hand]}',(text_loc_x, 70), cv2.FONT_HERSHEY_SIMPLEX,1.5, (0, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(img_list[i],f'action_time({label}):{action_time[hand]/fps:.2f}s',(text_loc_x, 120), cv2.FONT_HERSHEY_SIMPLEX,1.5, (0, 0, 0), 3, cv2.LINE_AA)
        do_mosaic(img_list[i], 700, 50, 500, 500,neighbor=30)
    out.write(img_list[i])

out.release()

In [ ]:
a = (1,2)
a = (3,4)